<h1>Predict Future Stock Price</h1>

In [238]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc

# For Training, Testing and building a Model
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing,model_selection

from datetime import datetime

plt.figure(figsize=(16,9))

In [239]:
days_in_future_predict = 1

In [240]:
conn_str = ('server=WIN-EEL3AK31AJF;database=DatapyDB;TRUSTED_CONNECTION=yes')
#df = pd.read_csv('D:\\data\\XBIT.csv')

In [241]:
conn = pyodbc.connect(r'DRIVER={ODBC Driver 13 for SQL Server};' + conn_str)

In [242]:
#sql = "SELECT * FROM dbo.Stock where StockSym = 'XBIT' AND StockDt < '2019-12-06'"

sql = "SELECT * FROM dbo.Stock where StockSym = 'XBIT' AND StockDt < '2019-12-06'"
df = pd.read_sql(sql,conn)

In [243]:
df.head()

In [244]:
df = df.drop(['StockSym'],axis=1)
df = df.drop(['StockDt'],axis=1)

In [245]:
df.head()

<h2>Add a derived column to the data frame</h2>

In [246]:
df['prediction'] = df[['StockAdjClose']].shift(-days_in_future_predict)

In [247]:
df.head()

In [248]:
df.tail(6)

In [249]:
df.describe()

In [250]:
df.dropna(inplace=True)

In [251]:
df.describe()

In [252]:
df.tail(3)

In [253]:
plt.figure(figsize=(16,9))
plt.scatter(df['StockAdjClose'],df['prediction'])

In [254]:
plt.figure(figsize=(16,9))
plt.plot(df['StockAdjClose'],color='red')
plt.plot(df['prediction'],color='green')

Read more on dataframe.drop
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [255]:
X = np.array(df.drop(['prediction'],1))

In [256]:
X

In [257]:
print(type(X))

In [258]:
len(X)

In [259]:
df.describe()

In [260]:
Y = np.array(df['prediction'])

In [261]:
Y

In [262]:
len(Y)

In [263]:
trainX,testX,trainY,testY = model_selection.train_test_split(X,Y,test_size=0.2,random_state=0)

<h2>As part our Supervised Machine Leaning exercise, for training purpose we are using 80% of the records from our dataframe. </h2> 

In [264]:
len(trainX)

In [265]:
len(testX)

In [266]:
len(trainY)

<h2>Let's build a Statistical Model using SciKit</h2>

In [267]:
stockPricePredictModel = DecisionTreeRegressor() 
#stockPricePredictModel = LinearRegression() 

# Using fit method we are passing all the 388 records stored in both trainX and trainY variables.
# This is the training step in the model building process.

stockPricePredictModel.fit(trainX, trainY)

# With our model trained with 80% of data from the dataframe
# we want to test and see how model scores on the remaining 20% of the data
# Remember in real-world you may be dealing with Terabyte or Petabyte of data

# Score method: Passing both testX and testY variables (they both have 20% of the data)
# RSquared value is always between 0 and 1 and 1 being high accurancy of prediction
# I'm multiplying this factor with 100 to represent the value in terms of percentage 
rSquaredFactore = (stockPricePredictModel.score(testX,testY)) * 100

#prediction = (clf.predict(X_prediction))

In [268]:
print(rSquaredFactore)

<h2>Let's see how our model predicts values for all the values in testX variable. Remeber testX has 20% of the data

Notice that I'm using predict method from the model. 
Tip: fit() to train, score() to get the R Squared Value and predict() to predict values  

In [269]:
allTestPrediction = stockPricePredictModel.predict(testX)

<h2>Let's compare and see how our model performed on predictions

In [270]:
# Creating a temp DataFrame
dfTemp = pd.DataFrame({'Actual Value in Test (20% of the data)':testY, 'Model Predicted as ':allTestPrediction})

In [271]:
dfTemp

In [272]:
plt.figure(figsize=(16,9))
plt.plot(testY,color="blue")
plt.plot(allTestPrediction,color="red")

In [273]:
sql = "SELECT TOP 1 * FROM dbo.Stock where StockSym = 'XBIT' AND StockDt < '2019-12-06' ORDER BY StockDt DESC"
dfLastRecordedTrade = pd.read_sql(sql,conn)

In [274]:
dfLastRecordedTrade.head()

In [275]:
dfLastRecordedTrade = dfLastRecordedTrade.drop(['StockSym'],axis=1)
dfLastRecordedTrade = dfLastRecordedTrade.drop(['StockDt'],axis=1)

In [276]:
dfLastRecordedTrade.head()

In [277]:
inputX = np.array(dfLastRecordedTrade)

In [278]:
inputX

In [279]:
futureStockPrice = stockPricePredictModel.predict(inputX)

In [280]:
print(futureStockPrice)